In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold

LR = LinearRegression()
kf = RepeatedKFold(n_splits=10, n_repeats=5)

# # MA |y-h(x)|
def MAE(y, y_hat, n):
    # sum = 0
    # for i in range(n):
    #     sum += abs(y_hat[i] - y[i])
    # return sum/n
    return np.absolute(np.subtract(y_hat,y)).mean()
# # MS (y-h(x))^2
def MSE(y, y_hat):
    return np.square(np.subtract(y,y_hat)).mean()
''' Implement Scaling Hyperparams'''
# # Huber
class Hub():
    def one(res, n, b):     # finding optimal tau
        huber_lf_1 = []
        huber_lf_2 = []
        tau_list = []
        count = 0
        for j in b:
            tau_list.append(j)
            for i in range (n):
                if res[i] < j:
                    huber_lf_1.append(count + ((res[i])**2)/2)
                else:
                    huber_lf_2.append(count + (j*abs(res[i])) - ((j**2)/2))
            count += 1
        if huber_lf_1 == False:
            ix = int(str(int(min(huber_lf_2)))[:1])
            return (tau_list[ix])
        elif huber_lf_2 == False:
            ix = int(str(int(min(huber_lf_1)))[:1])
            return (tau_list[ix])

        else:
            if min(huber_lf_2) > min(huber_lf_1):
                ix = int(str(int(min(huber_lf_1)))[:1])
                return (tau_list[ix])
            else:
                ix = int(str(int(min(huber_lf_2)))[:1])
                return (tau_list[ix])

    def two(res, n, b):     # testing tau value
        huber_lf_1 = []
        huber_lf_2 = []
        for i in range (n):
            if res[i] < b:
                huber_lf_1.append(((res[i])**2)/2)
            else:
                huber_lf_2.append((b*abs(res[i])) - ((b**2)/2))
        if min(huber_lf_2) > min(huber_lf_1):
            return min(huber_lf_1)
        else:
            return min(huber_lf_2)
# # Cauchy
class Cau():
    def one(res, n, b):      # finding optimal tau
        cauchy_lf = []
        tau_list = []
        count = 0
        for j in b:
            tau_list.append(j)
            for i in range (n):
                cauchy_lf.append(count + ((j**2) * np.log(1+(((res[i])**2)/(j**2)))))
            count += 1

        ix = int(str(int(min(cauchy_lf)))[:1])
        return (tau_list[ix])
        
    def two(res, n, b):     # testing tau value
        cauchy_lf = []
        for i in range (n):
            cauchy_lf.append(((b**2) * np.log(1+(((res[i])**2)/(b**2)))))
        return min(cauchy_lf)

### 3-step sliding frame dataset ###

In [2]:
# Positive (3, 0.25)

df_3step = pd.read_csv("train_P.csv", usecols=['RUC', 'kappa', 'sliding_frame'], index_col=False)
df_3step = df_3step[(df_3step["sliding_frame"] == 3) & (df_3step["kappa"] == 0.5)]
p3s_ruc = df_3step[df_3step["RUC"] > 0]
n_p = p3s_ruc.shape[0]

x1 = np.array(range(0,n_p)).reshape(-1,1)
y1 = np.array(p3s_ruc["RUC"])
LR.fit(x1,y1)
y_hat1 = LR.predict(x1)
res = y1-y_hat1

abs_ruc_p = MAE(y1,y_hat1,n_p)
sq_ruc_p = MSE(y1,y_hat1)

b_min = min(y1)
b_max = max(y1)
alpha = np.median(y1)
b_p = np.arange(b_min,b_max,alpha)

scores_hub = []
scores_cauc = []

for train_ix, test_ix in kf.split(res):
    y_train = res[train_ix]
    scores_hub.append(Hub.one(y_train, train_ix.shape[0], b_p))
    scores_cauc.append(Cau.one(y_train, train_ix.shape[0], b_p))
tmin_h = np.mean(scores_hub)
tmin_c= np.mean(scores_cauc)

print(sq_ruc_p,Hub.two(res,n_p,tmin_h))

# print("\nHuber ERV: ", hub_ruc_p, "\nCauchy ERV: ",cau_ruc_p, "\nMAE:" , abs_ruc_p,"\nMSE:" , sq_ruc_p)


KeyboardInterrupt: 

In [ ]:
# Negative (3,0.25)

df_3step = pd.read_csv("train_P.csv", usecols=['RUC', 'kappa', 'sliding_frame'], index_col=False)
df_3step = df_3step[(df_3step["sliding_frame"] == 3) & (df_3step["kappa"] == 0.5)]
n3s_ruc = df_3step[df_3step["RUC"] < 0]
n_n = n3s_ruc.shape[0]

x2 = np.array(range(0,n_n)).reshape(-1,1)
y2 = np.array(n3s_ruc["RUC"])
LR.fit(x2,y2)
y_hat2 = LR.predict(x2)
res = y2-y_hat2

abs_ruc_n = MAE(y2,y_hat2,n_n)
sq_ruc_n = MSE(y2,y_hat2)

b_min = max(y2)
b_max = min(y2)
alpha = np.median(y2)
b_n = np.arange(b_min,b_max,alpha)

scores_hub = []
scores_cauc = []

for train_ix, test_ix in kf.split(res):
    y_train = res[train_ix]
    scores_hub.append(Hub.one(y_train, train_ix.shape[0], b_n))
    scores_cauc.append(Cau.one(y_train, train_ix.shape[0], b_n))
tmin_h = np.mean(scores_hub)
tmin_c= np.mean(scores_cauc)
print(sq_ruc_n,Hub.two(res,n_n,tmin_h))

# print("\nHuber ERV: ", hub_ruc_p, "\nCauchy ERV: ",cau_ruc_p, "\nMAE:" , abs_ruc_p,"\nMSE:" , sq_ruc_p)


0.0029590027335597104 -1.0833798524344104e-07


In [ ]:
class first():
    if 1+1==2:
        print(8)
    def second():
        print(9)

first.second()

8
9


### 7-step sliding frame dataset ###

In [ ]:
# df_7step = pd.read_csv("train_P.csv", usecols=['RUC', 'kappa', 'sliding_frame'])

# df_7step = df_7step[(df_3step["sliding_frame"] == 7) & (df_7step["kappa"] <= 0.75)]
# p7s_ruc = df_7step[df_7step["RUC"] > 0]
# n7s_ruc = df_7step[df_7step["RUC"] < 0]